import data

In [1]:
import pickle
import tensorflow as tf
import numpy as np
from PIL import Image as im
import math
from scipy import io

In [2]:

search_size=5
index_center_pixel=int((search_size*search_size-1)/2) #center pixel index
sample_number=50 #batch index
cutoff=5
classes_number=20
h=64
w=1024
total_number_of_points=130000
neibor_number=5
file = open ("labels2d.bin","rb")
labels2d= pickle.load(file)
file.close()
file = open ("depth2d.bin","rb")
depth2d= pickle.load(file)
file.close()
file = open ("depth3d.bin","rb")
depth3d= pickle.load(file)
file.close()
file = open ("points_2d_mapping.bin","rb")
points_2d_mapping= pickle.load(file)
file.close()
labels2d= labels2d.numpy()
depth3d= depth3d.numpy()
depth2d= depth2d.numpy()
points_2d_mapping= points_2d_mapping.numpy()
depth2d=depth2d[sample_number,:,:]
depth3d=depth3d[sample_number,:]
points_2d_mapping=points_2d_mapping[sample_number,:]
print (labels2d[1,2,33,:])
labels2d=labels2d[sample_number,:,:,:]


# show the sample image
im.fromarray(np.uint8(depth2d*10)).save('depth2dImage.png')




#piture extention for the window funtion to work
# padding
depth2d=np.insert(depth2d,1024,depth2d[:,0],axis=1)
depth2d=np.insert(depth2d,1025,depth2d[:,1],axis=1)
depth2d=np.insert(depth2d,1026,depth2d[:,2],axis=1)
depth2d=np.insert(depth2d,1027,depth2d[:,3],axis=1)
depth2d=np.insert(depth2d,64,np.zeros(1028),axis=0)
depth2d=np.insert(depth2d,65,np.zeros(1028),axis=0)
depth2d=np.insert(depth2d,66,np.zeros(1028),axis=0)
depth2d=np.insert(depth2d,67,np.zeros(1028),axis=0)


label2dnew=np.zeros((h,w))
for ww in range (1024):
        for hh in range (64):
                for j in range(20):
                        if 1==labels2d[hh,ww,j]:
                            label2dnew[hh,ww]=j
#save the label image
data = im.fromarray(np.uint8(label2dnew*5))
data.save('label2DtoImage.png')

label2dnew=np.insert(label2dnew,1024,label2dnew[:,0],axis=1)
label2dnew=np.insert(label2dnew,1025,label2dnew[:,1],axis=1)
label2dnew=np.insert(label2dnew,1026,label2dnew[:,2],axis=1)
label2dnew=np.insert(label2dnew,1027,label2dnew[:,3],axis=1)
label2dnew=np.insert(label2dnew,64,np.zeros((1028,1)),axis=0)
label2dnew=np.insert(label2dnew,65,np.zeros((1028,1)),axis=0)
label2dnew=np.insert(label2dnew,66,np.zeros((1028,1)),axis=0)
label2dnew=np.insert(label2dnew,67,np.zeros((1028,1)),axis=0)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


gauaain_kernel

In [ ]:
def get_gaussian_kernel(kernelsize=3, sigma=2):
    xy_grid = np.indices((kernelsize, kernelsize))

    mean = (kernelsize - 1) / 2.0
    variance = sigma**2

    gaussian = (1 / (2 * np.pi * variance)) * np.exp(
        -np.sum((xy_grid - mean) ** 2, axis=0) / (2 * variance)
    )
    gaussian = gaussian / np.sum(gaussian)
    return gaussian

G_prim=get_gaussian_kernel(search_size,1)

Get S^2 neighbors N_prim[h*w,S^2] for each pixel

In [ ]:
N_prim=np.zeros((w*h,search_size*search_size))
for v in range (h):
    for u in range (w):
        for i in range (search_size):
            for j in range (search_size):
             N_prim [v*w+u][i*search_size+j]=depth2d[v+j-2][u+i-2]

Get neighbors N for each point &  Fill in real point ranges 

In [ ]:
N=np.zeros((total_number_of_points,search_size*search_size))

for point_index, pixel_index_uv in enumerate(points_2d_mapping):
    N[point_index, :] = N_prim[pixel_index_uv, :]
    N[point_index, index_center_pixel]=depth3d[point_index]



 Label neighbors L_prim for each pixel

In [ ]:
L_prim=np.zeros((w*h,search_size*search_size))
for v in range (h):
    for u in range (w):
        for i in range (search_size):
            for j in range (search_size):
             L_prim [v*w+u,i*search_size+j]=label2dnew[v+j-2,u+i-2]

Get label neighbors L for each point 

In [ ]:
L=np.zeros((total_number_of_points,search_size*search_size))


for point_index, pixel_index_uv in enumerate(points_2d_mapping):
    L[point_index, :] = L_prim[pixel_index_uv, :]
    n +=1   

 Distances to neighbors D for each point

In [ ]:
D=np.zeros((total_number_of_points,search_size*search_size))
for i in range (total_number_of_points):
    D[i,:]=np.absolute([N[i,:]-N[i,index_center_pixel]])    

Compute inverse Gaussian Kernel 

In [ ]:
G=1-np.max(G_prim)*G_prim
G = G.flatten()

Weight neighbors with inverse Gaussian kernel

In [ ]:
for i in range (total_number_of_points):
    D [i,:]=D [i,:]*G[:]

Find k-nearest neighbors S for each point

In [ ]:
D_sorted=np.zeros((total_number_of_points,search_size*search_size))
D_sorted=np.argsort(D,1)
S_sorted=D_sorted[:,0:neibor_number]

 Gather votes

In [ ]:
L_knn=np.zeros((total_number_of_points,neibor_number))
for i in range (total_number_of_points):
    n=0
    for j in S_sorted[i,:]:
        if D[i,j] > cutoff:
            L_knn[i,n]=classes_number+1
        else :
            L_knn[i,n]=L[i,j]
        n +=1

 Accumulate votes

In [ ]:
Votes=np.zeros((total_number_of_points,classes_number))
for i in range (total_number_of_points):
    for j in range (neibor_number):
        if L_knn[i,j]<classes_number:
            Votes[i,int(L_knn[i,j])] +=1

 Find maximum consensus

In [ ]:
L_cons=np.zeros(total_number_of_points)
for i in range(total_number_of_points):
    L_cons[i]=np.argmax(Votes[i,:])

In [ ]:
file = open ("Labes3D","wb")
pickle.dump(np.uint8(L_cons*100),file)
file.close()

make a point cloud data for matlab

In [ ]:
def pointcloud_maker(points_2d_mapping,depth3d):
    point_cloud = np.zeros((total_number_of_points,3))
    
    for i in range (total_number_of_points):
        point=points_2d_mapping[i]
        r = depth3d[i]
        z=math.floor(point/1024)
        theta=((2*math.pi)/1024)*math.floor(point%1024)
        x=r*math.sin(theta)
        y=r*math.cos(theta)
        point_cloud[i,0]=x
        point_cloud[i,1]=y
        point_cloud[i,2]=math.sin((1-z/64)*0.49-0.45)*r
    

    return point_cloud



PT=pointcloud_maker(points_2d_mapping,depth3d)

io.savemat('PTpoints.mat', {"data": PT })
io.savemat('PTlabels.mat', {"labels": L_cons })